In [31]:
#import library for api calls
import requests
import json
import pandas as pd
import numpy as np
import re
import pickle
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

{'paperId': '62d1a3137b01a69443bebf4d92c1990ec512a6a1', 'url': 'https://www.semanticscholar.org/paper/62d1a3137b01a69443bebf4d92c1990ec512a6a1', 'title': 'Extracting Training Data from Large Language Models', 'citationCount': 402, 'influentialCitationCount': 35, 'authors': [{'authorId': '2483738', 'name': 'Nicholas Carlini'}, {'authorId': '2444919', 'name': 'Florian Tramèr'}, {'authorId': '145217343', 'name': 'Eric Wallace'}, {'authorId': '40844378', 'name': 'Matthew Jagielski'}, {'authorId': '1404060687', 'name': 'Ariel Herbert-Voss'}, {'authorId': '2119194377', 'name': 'Katherine Lee'}, {'authorId': '145625142', 'name': 'Adam Roberts'}, {'authorId': '31035595', 'name': 'Tom B. Brown'}, {'authorId': '143711382', 'name': 'D. Song'}, {'authorId': '1758110', 'name': 'Ú. Erlingsson'}, {'authorId': '3046437', 'name': 'Alina Oprea'}, {'authorId': '2402716', 'name': 'Colin Raffel'}]}


In [327]:

#given a semantic scholar paper ID, get the authors with their affiliations
def get_authors(paper_id):
    request_url = 'https://api.semanticscholar.org/graph/v1/paper/{}/authors?fields=name,affiliations'.format(paper_id)

    response = requests.get(request_url)
    response_json = response.json()
    while 'message' in response_json.keys():
        print(response_json['message'])
        #wait 5 minutes and try again
        #print the time
        print(time.strftime("%H:%M:%S", time.localtime()))
        print("waiting five minutes")
        #print the time this will resume
        print(time.strftime("%H:%M:%S", time.localtime(time.time() + 310)))
        time.sleep(310)
        response = requests.get(request_url)
        response_json = response.json()
    authors = response_json['data']
    return authors

#get_authors('62d1a3137b01a69443bebf4d92c1990ec512a6a1')



In [44]:
#define a function to get the papers for a given year, taking string search_query and integer paper_year as input
def get_papers(search_query: str, paper_year: int):
    request_url = 'https://api.semanticscholar.org/graph/v1/paper/search?query=' + search_query + '&year=' + str(paper_year) + '&offset=0&limit=100&fields=url,authors,title,influentialCitationCount,citationCount'
    response = requests.get(request_url)
    response_json = response.json()

    #print(response_json['data'])
    return(response_json['data'])

In [311]:
year_range = range(2012,2022)

In [51]:
#loop through years, get papers for that year, then get authors for each paper
#return a data frame, one row for each author*paper, listing the paper's ID, author's name, and affiliation
#return another data frame, one row for each paper, listing the paper's ID, title, year, influential citation count, and citation count
#do this serially in order to avoid hitting the API rate limit

papers_by_year_dict = {}
for year in year_range:
    print(year)
    papers_for_year = get_papers('large "language model"', year)
    papers_by_year_dict[year]=papers_for_year
    print(len(papers_for_year))
    for paper in papers_for_year:
        print(paper['citationCount'],end=", ")
    print("\n")
    for paper in papers_for_year:
        print(paper['influentialCitationCount'],end=", ")
    

2012
100
25, 14, 9, 1, 28, 19, 18, 6, 6, 3, 3, 2, 2, 2, 5, 1, 3, 2780, 57, 51, 28, 10, 4, 4, 2, 19, 256, 8, 8, 7, 3, 65, 1723, 53, 2, 239, 47, 947, 34, 60, 72, 24, 34, 40, 58, 49, 53, 39, 33, 21, 34, 19, 34, 28, 112, 29, 79, 79, 31, 15, 21, 73, 27, 66, 57, 56, 16, 21, 50, 20, 9, 9, 8, 9, 13, 16, 41, 12, 347, 14, 10, 3, 3, 331, 9, 8, 8, 34, 33, 4, 7, 7, 3, 5, 4, 31, 2, 2, 6, 6, 

1, 0, 1, 0, 1, 3, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 142, 2, 4, 2, 1, 0, 0, 0, 2, 10, 1, 2, 0, 0, 6, 117, 3, 0, 16, 5, 87, 3, 8, 2, 2, 3, 7, 4, 6, 5, 2, 0, 0, 2, 0, 0, 4, 3, 0, 8, 12, 2, 0, 4, 0, 1, 1, 2, 4, 3, 1, 2, 3, 2, 1, 0, 0, 1, 0, 4, 1, 25, 0, 0, 0, 0, 15, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2013
100
1, 584, 46, 36, 23, 11, 6, 8, 3, 2, 1, 2, 1, 10, 271, 953, 60, 33, 4, 6, 276, 2, 119, 224, 8, 7, 7, 6, 9, 133, 4, 91, 2, 147, 134, 48, 4, 45, 37, 44, 41, 29, 30, 136, 41, 101, 566, 30, 432, 22, 19, 16, 15, 11, 465, 26, 24, 8, 11, 59, 14, 16, 9, 6, 6, 20, 8, 39, 425, 9, 11, 13, 10, 7, 6, 39, 9, 2, 8

In [141]:
year = 2021
papers_for_year = get_papers('large "language model"', year)
papers_by_year_dict[year]=papers_for_year
print(len(papers_for_year))
for paper in papers_for_year:
    print(paper['citationCount'],end=", ")
print("\n")
for paper in papers_for_year:
    print(paper['influentialCitationCount'],end=", ")


100
40, 7, 27, 450, 52, 48, 19, 144, 15, 93, 118, 40, 3, 5, 8, 6, 5, 5, 16, 95, 22, 26, 18, 24, 16, 20, 185, 20, 17, 17, 6, 12, 14, 10, 6, 9, 6, 6, 8, 4, 3, 5, 4, 4, 5, 5, 5, 5, 4, 34, 68, 17, 1, 93, 94, 68, 24, 50, 20, 13, 4, 10, 13, 9, 16, 6, 37, 22, 33, 5, 5, 29, 11, 8, 6, 6, 78, 8, 274, 244, 49, 42, 145, 46, 11, 21, 6, 28, 39, 94, 8, 10, 40, 62, 6, 60, 58, 26, 50, 52, 

11, 0, 1, 129, 15, 5, 5, 32, 1, 6, 10, 4, 0, 0, 2, 0, 1, 0, 1, 14, 7, 1, 1, 0, 0, 0, 74, 2, 1, 2, 0, 3, 1, 1, 2, 0, 3, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 6, 24, 1, 0, 21, 13, 12, 1, 4, 3, 3, 1, 1, 0, 0, 4, 0, 5, 8, 10, 0, 0, 0, 2, 3, 1, 0, 16, 1, 80, 26, 4, 7, 24, 5, 2, 6, 1, 6, 2, 18, 1, 0, 9, 3, 0, 10, 6, 2, 15, 9, 

In [137]:
#OK, now, for each year, get the authors for each paper wit htheir affilaitions
#let's start with just 2012
#this will take a while.
authors_by_paper_dict = {}
for year in year_range:
    print(year)
    papers_for_year = papers_by_year_dict[year]
    for i, paper in enumerate(papers_for_year):
        print(".",end="")
        #print(paper['title'])
        paper_authors = get_authors(paper['paperId'])
        # if i % 10 == 0:
        #     print(paper['title'])
        authors_by_paper_dict[paper['paperId']]=paper_authors
    print("\n\n")

2012
....................................................................................................


2013
....................................................................................................


2014
................................................Too Many Requests
19:41:38
waiting five minutes
19:46:38
....................................................


2015
....................................................................................................


2016
....................................................................................................


2017
..............................Too Many Requests
19:47:37
waiting five minutes
19:52:37
Too Many Requests
19:52:38
waiting five minutes
19:57:38
......................................................................


2018
....................................................................................................


2019
....................................................................

In [142]:
#OK, now, for each year, get the authors for each paper wit htheir affilaitions
#let's start with just 2012
#this will take a while.
#authors_by_paper_dict = {}
year=2021
print(year)
papers_for_year = papers_by_year_dict[year]
for i, paper in enumerate(papers_for_year):
    print(".",end="")
    #print(paper['title'])
    paper_authors = get_authors(paper['paperId'])
    # if i % 10 == 0:
    #     print(paper['title'])
    authors_by_paper_dict[paper['paperId']]=paper_authors
print("\n\n")

2021
....................................................................................................




In [143]:
#save the authors_by_paper_dict to a file

with open('../../data/authors_by_paper_dict.pickle', 'wb') as handle:
    pickle.dump(authors_by_paper_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [302]:
def convert_author_affiliation_dict_to_df(authors_by_paper_dict):
    #now let's put this into a dataframe
    #one row for each author*paper, listing the paper's ID, author's name, and affiliation
    author_df = pd.DataFrame(columns=['paper_id', 'author_name', 'author_affiliation'])
    for paper_id in authors_by_paper_dict.keys():
        authors= authors_by_paper_dict[paper_id]
        for a_i, author in enumerate(authors):
            #create a data frame holding the paper ID, author name, and affiliation
            if author['affiliations'] is None or len(author['affiliations'])==0:
                #author['affiliations'] = None
                author_row = pd.DataFrame({'paper_id':[paper_id], 'authorId':[author['authorId']], 'author_name':[author['name']],'author_precedence':[a_i]})
            else:
                a_count = len(author['affiliations'])
                # if a_count>1:
                #     print("more than one affiliation")
                #     print(paper_id, author['name'], author['affiliations'])
                author_row = pd.DataFrame({'paper_id':[paper_id]*a_count, 'authorId':[author['authorId']]*a_count, 'author_name':[author['name']]*a_count, 'author_affiliation':author['affiliations'],'author_precedence':[a_i]*a_count})
            author_df = author_df.append(author_row, ignore_index=True)
        
            #print(paper_id, author['name'], author['affiliations'])

            #print(author['name'], author['affiliations'])
    author_df.author_precedence=author_df.author_precedence.astype(int)
    return author_df

In [291]:
#def convert_papers_json_to_df(papers_json)
#convert list of dictionaries to a data frame
def convert_papers_json_to_df(papers_json):
    papers_df = pd.DataFrame(papers_json)
    #remove authors from the data frame
    papers_df = papers_df.drop(columns=['authors'])    
    return papers_df

#convert the papers_by_year_dict to a data frame
papers_df_list = []
for year in papers_by_year_dict.keys():
    papers_for_year = papers_by_year_dict[year]
    papers_for_year_df = convert_papers_json_to_df(papers_for_year)
    papers_for_year_df['year'] = year
    papers_df_list = papers_df_list + [papers_for_year_df]

papers_df = pd.concat(papers_df_list, ignore_index=True)

In [290]:
#pickle papers_for_year
with open('data/papers_by_year_dict.pickle', 'wb') as handle:
    pickle.dump(papers_by_year_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

OK, now I think we almost have enough to put this together...

In [303]:
author_df = convert_author_affiliation_dict_to_df(authors_by_paper_dict)

In [304]:
#print authors with affiliations
author_df[author_df['author_affiliation'].notnull()]

,paper_id,author_name,author_affiliation,authorId,author_precedence
76,6226e7dcee9e050face0caf852de37a9c2937812,Aline Villavicencio,"University of Sheffield, Federal University of...",145585242,0
84,2c898abb4d7f1c1793f99632396afd9f3d149ae2,Yasuhisa Fujii,Google,1851865,4
93,d20744d41de25c3181f3b3fcb749531a6ec9e76d,Sandra Kübler,Indiana University,1804668,0
150,0ce317d84086b8885ddbc7923ec00bedb64ab6dc,François Yvon,"LIMSI, CNRS",1846431,2
172,beb1226c89615d2b092b9230c4df5c496a2a76eb,Haizhou Li,National University of Singapore,1711271,2
...,...,...,...,...,...
4338,26e3d58181724f9ef77973ff0f65bac06e499fec,Yu Yan,microsoft.com,145967727,2
4358,c7d8b6180b52f9be891fec1223b7f561770b0206,Robert Schwarzenberg,German Research Center for Artificial Intellig...,1683203,1
4365,e1a3e6856b6ac6af3600b5954392e5368603fd1b,Huan Yang,Microsoft Research,46402216,5
4369,b53c386b7c65af80905dc05a9b27e98e03324739,Viet Dac Lai,University of Oregon,1405279380,1


In [305]:

def code_affiliation(affiliation):
    #check if affiliation contains the word 'University'
    #do a fuzzy match using fuzzywuzzy on whether affiliation contains 'University'
    #if it does, return 'Academic'
    
    #fuzzywuzzy match
    #print(affiliation)
    if affiliation is np.nan:
        return affiliation

    al=affiliation.lower()
    
    if fuzz.partial_ratio("university",al)>=80:
        return 'Academic'
    if fuzz.partial_ratio("Institute of Technology".lower(),affiliation)>=90:
        return 'Academic'
    misc_academic_names = [
        "Univ.","Nara Institute of Science and Technology (NAIST)",
         "Univ. Grenoble Alpes","MIT","LMU Munich","UCLA","UNC Chapel Hill","Georgia Tech",
         "Stanford","Brooklyn College"]
    #check to see if any of the misc academic names are in the affiliation
    for name in misc_academic_names:
        if name.lower() in al:
            return 'Academic'

    #now do corporate affiliations
    corporate_fuzzy_matches = [
        "Google","Facebook","Microsoft","DeepMind","Meta AI",
        "Amazon","IBM","Apple","Baidu","Alibaba","JD.com","Tencent",
        "Huawei","Salesforce"
    ]
    for cfm in corporate_fuzzy_matches:
        if fuzz.partial_ratio(cfm,affiliation)>=90:
            return 'For-profit corporation'

    #non-profit institutes
    nonprofit_fuzzy_matches = ["OpenAI","Allen Institute"]
    for fuzzy_match_str in nonprofit_fuzzy_matches:
        if fuzz.partial_ratio(fuzzy_match_str,affiliation)>=90:
            return 'Non-profit organization'


    return 'Undetermined'

In [306]:
author_df['affiliation_type'] = [code_affiliation(aa) for aa in author_df.author_affiliation]

In [307]:
author_df['affiliation_type'].value_counts()

Academic                   263
For-profit corporation     116
Undetermined                84
Non-profit organization     26
Name: affiliation_type, dtype: int64

In [308]:
author_df.loc[author_df['author_precedence']==0,'affiliation_type'].value_counts()

Academic                   82
Undetermined               21
For-profit corporation     20
Non-profit organization     7
Name: affiliation_type, dtype: int64

In [276]:
#get other affiliations
other_affiliations = author_df['author_affiliation'][author_df['affiliation_type']=='Undetermined']
#sort by frequency, then print in that order
sorted_affiliations = other_affiliations.value_counts().sort_values(ascending=False)
print(sorted_affiliations[sorted_affiliations>2])
print("appearing twice:")
print(", ".join(sorted_affiliations[sorted_affiliations==2].index))
print("appearing once")
print(", ".join(sorted_affiliations[sorted_affiliations==1].index))

Inria    3
Name: author_affiliation, dtype: int64
appearing twice:
EPFL, Waymo, Samsung Electronics, ETH Zürich, UC Davis, Department of Computer Science, CMU LTI, Samsung Research, National Institute of Information & Communications Technology (NICT), L3S Research Center, NICT
appearing once
ASAPP Inc, FAIR, Symanto, Square Inc, Adobe, ETH Zurich, Qualcomm, Reveal, Technion, NHK, Inflection AI, HuggingFace Inc., Adobe Research, UC Berkeley, Hasso Plattner Institute, CRIM, UMASS Amherst, MDC Berlin, Qatar Computing Research Institute, ZBW - Leibniz Information Centre for Economics, Fluent.ai Inc., Sulzer GmbH, Advanced Digital Sciences Center Singapore, Truveta, German Research Center for Artificial Intelligence (DFKI), Nuance Communications Inc., Seagate, Worcester Polytechnic Institute, National Institute of Advanced Industrial Science and Technology, ObEN, Idiap Research Institute, Ex: Uber Technologies Inc, Honda Research Institute Japan, rippleAI, KAIST LK Lab, École de Technologie

Let's code these manually in a spreadsheet, then come back to here.

In [279]:
#save in a format comaptible in microsoft excel
author_df.to_csv("data/affiliations.csv",encoding='utf-8')

In [294]:
papers_df.to_csv("data/papers_all_df.csv",encoding='utf-8')

now determine whether the author affiliation is a university or not. with only 416 values overall this will be best coded manually in a sheet

## more data

We're going to try a few more search terms to widen our dataset.




In [315]:
papers_by_year_multiterm_dict = {}
papers_by_year_multiterm_dict['large "language model"'] = papers_by_year_dict
authors_by_paper_by_term_dict = {}
authors_by_paper_by_term_dict['large "language model"'] = authors_by_paper_dict

In [312]:
#loop through years, get papers for that year, then get authors for each paper
#return a data frame, one row for each author*paper, listing the paper's ID, author's name, and affiliation
#return another data frame, one row for each paper, listing the paper's ID, title, year, influential citation count, and citation count
#do this serially in order to avoid hitting the API rate limit



term='deep learning'
papers_by_year_for_term_dict = {}
for year in year_range:
    print(year)
    papers_for_year = get_papers(term, year)
    papers_by_year_for_term_dict[year]=papers_for_year
    print(len(papers_for_year))
    for paper in papers_for_year:
        print(paper['citationCount'],end=", ")
    print("\n")
    for paper in papers_for_year:
        print(paper['influentialCitationCount'],end=", ")
    
papers_by_year_multiterm_dict[term]=papers_by_year_for_term_dict

2012
100
5, 9593, 614, 7769, 504, 1561, 245, 153, 170, 193, 102, 186, 126, 87, 167, 39, 3596, 50, 53, 62, 29, 30, 30, 3215, 73, 23, 24, 38, 24, 20, 30, 13, 6, 7, 11, 9, 22, 6, 8, 6, 7, 4, 5, 3, 3, 3, 19, 16, 16, 2, 2, 7, 14, 1, 16, 3, 3, 6, 2, 5, 3, 2, 3, 2, 0, 0, 424, 404, 418, 1825, 184, 202, 146, 110, 194, 1262, 80, 73, 45, 52, 61, 800, 40, 837, 66, 23, 32, 31, 14, 27, 17, 619, 36, 12, 12, 8, 6, 7, 4, 9, 

0, 527, 49, 989, 33, 127, 18, 9, 6, 13, 6, 9, 5, 2, 13, 4, 172, 3, 2, 3, 5, 0, 0, 323, 1, 0, 0, 2, 1, 0, 1, 0, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 38, 23, 46, 149, 31, 15, 5, 11, 7, 43, 15, 7, 4, 3, 2, 37, 1, 192, 5, 2, 2, 4, 0, 3, 1, 47, 3, 0, 0, 0, 1, 1, 0, 0, 2013
100
3899, 8021, 1271, 1094, 744, 498, 705, 614, 723, 593, 1994, 311, 1601, 386, 374, 353, 334, 462, 374, 192, 1403, 157, 212, 281, 154, 158, 252, 88, 151, 70, 47, 141, 53, 41, 48, 57, 36, 82, 101, 77, 35, 114, 136, 35, 75, 81, 60, 42, 33, 34, 100, 32, 2

In [320]:
papers_already_seen = papers_df['paperId'].unique()

In [328]:
term='deep learning'
papers_for_this_term_dict = papers_by_year_multiterm_dict[term]
#OK, now, for each year, get the authors for each paper with their affilaitions
#let's start with just 2012
#this will take a while.
authors_by_paper_for_term_dict = {}
for year in year_range:
    print(year)
    papers_for_year = papers_for_this_term_dict[year]
    for i, paper in enumerate(papers_for_year):
        print(".",end="")
        #print(paper['title'])
        cur_paper_id=paper['paperId']
        if cur_paper_id in papers_already_seen:
            print("already seen this paper")
            continue
        
        paper_authors = get_authors(cur_paper_id)
        print("*",end="")
        # if i % 10 == 0:
        #     print(paper['title'])
        authors_by_paper_for_term_dict[paper['paperId']]=paper_authors
    print("\n\n")

authors_by_paper_by_term_dict[term]=authors_by_paper_for_term_dict

2012
.Too Many Requests
17:30:10
waiting five minutes
17:35:20
*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*


2013
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*


2014
.*.*.*.Too Many Requests
17:36:15
waiting five minutes
17:41:25
*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*


2015
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*


2016
.*.*.*.*.*.already seen this paper
.*.already see

The hierarchial design is a bit confusing, but it means I don't have to go back and re-run all the requests each time I want to add more data to the set, and can re-use the code I wrote when I just had a single search term.

In [349]:
#def convert_papers_json_to_df(papers_json)
#convert list of dictionaries to a data frame
def convert_papers_json_to_df(papers_json):
    papers_df = pd.DataFrame(papers_json)
    #remove authors from the data frame
    papers_df = papers_df.drop(columns=['authors'])    
    return papers_df

#convert the papers_by_year_dict to a data frame
papers_df_list = []
author_df_list = []
for term in papers_by_year_multiterm_dict.keys():
    print(term)
    papers_by_year_dict = papers_by_year_multiterm_dict[term]
    #convert papers hierarchical dict to a data frame
    for year in papers_by_year_dict.keys():
        print(year)
        papers_for_year = papers_by_year_dict[year]
        papers_for_year_df = convert_papers_json_to_df(papers_for_year)
        papers_for_year_df['year'] = year
        papers_for_year_df['term'] = term
        papers_df_list = papers_df_list + [papers_for_year_df]
    #convert authors hierarchical dict to a data frame
    author_df_list = author_df_list + [convert_author_affiliation_dict_to_df(authors_by_paper_by_term_dict[term])]

papers_df = pd.concat(papers_df_list, ignore_index=True)
author_df = pd.concat(author_df_list, ignore_index=True)

author_df['affiliation_type'] = [code_affiliation(aa) for aa in author_df.author_affiliation]

large "language model"
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
deep learning
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [351]:
#save in a format comaptible in microsoft excel
author_df.to_csv("data/affiliations_extended.csv",encoding='utf-8')
papers_df.to_csv("data/papers_all_df_extended.csv",encoding='utf-8')

In [352]:
#save the authors_by_paper_dict to a file

with open('data/authors_by_paper_dict_extended.pickle', 'wb') as handle:
    pickle.dump(authors_by_paper_by_term_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#pickle papers_for_year
with open('data/papers_by_year_dict_extended.pickle', 'wb') as handle:
    pickle.dump(papers_by_year_multiterm_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)